The following exercise is to create a test function that calculates the inner product of a noisy stabilizer state with another stabilizer state, and then calculates the derivative of the inner product with respect to a noise probability `p`. The exercise utilizes the `QuantumClifford` and `StochasticAD` packages.

In [1]:
using Pkg

# add `QuantumClifford` package
Pkg.add("QuantumClifford")
using QuantumClifford

# add `StochasticAD` package
Pkg.add("StochasticAD")
using StochasticAD

# `Distributions` package
Pkg.add("Distributions")
using Distributions

# `LinearAlgebra` package
Pkg.add("LinearAlgebra")
using LinearAlgebra

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [2]:
function consistent_stab(a)
    #= A function to test if a given stabilizer tableau represents a unique 
       quantum state. Not checking for Hermiticity here, can add this in an 
       updated version of the function =#
    
    # First check the mutual commutivity of rows:
    
    for i in 1:(nqubits(a) - 1)
        for j in (i+1):nqubits(a)
            if comm(a[i],a[j]) != 0
                return "Error: not all generators commute"
            end
        end
    end
    
    # Then check if the rank is full (determining that the rows are 
    # all independent and that the state is uniquely determined):
    
    if canonicalize!(a, ranks=true)[3] != nqubits(a)
        return "Error: state is not uniquely determined"
    end
    
return nothing
end

consistent_stab (generic function with 1 method)

In [3]:
function NoisyStabilizerOverlap(p, s1 = ghz(), s2 = one(Stabilizer, nqubits(s1)))
    # check if the s1 is a true stabilizer state; if not, print error message
    if consistent_stab(s1) != nothing
        return "Error: $s1 does not represent a consistent stabilizer state"
    end
    
    # check if the s2 is a true stabilizer state; if not, print error message
    if consistent_stab(s2) != nothing
        return "Error: $s2 does not represent a consistent stabilizer state"
    end
    
    # apply iid depolarization noise, and then calculate the overlap between 
    # the noisy s1 and s2
    function depolarization_noise(g)
        temp = copy(s1)
        apply!(temp, NoiseOpAll(UnbiasedUncorrelatedNoise(g/3)))
        overlap = dot(temp, s2)
        return overlap
    end
    
    # apply iid depolarization noise and calculate overlap many times 
    # to get derivative estimate
    samples = [derivative_estimate(depolarization_noise, p) for i in 1:1000] #samples
    derivative = mean(samples)
    uncertainty = std(samples) / sqrt(1000)
    println("derivative of 𝔼[dot(s1,s2)] = $derivative ± $uncertainty")
end

NoisyStabilizerOverlap (generic function with 3 methods)

We can now apply the `NoisyStabilizerOverlap` function for several different values of the noise probability `p` and see how the derivative is modified:

In [5]:
s1 = random_stabilizer(9)
s2 = random_stabilizer(9)

for i in 1:50
    NoisyStabilizerOverlap(.02*i,s1,s2)
end

derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0
derivative of 𝔼[dot(s1,s2)] = 0.0 ± 0.0


We can see that after trying the `NoisyStabilizerOverlap` function for many different values of the probability for iid depolarization noise, the derivative of the overlap always comes out to 0. I think there is some issue with how the `StochasticAD` handles my implementation of the inner product with random noise which does not implement the propagation of the $ \epsilon $ parameter. I have tried many different variations without success, including applying simple iid bitflip noise by cycling through a random variable `rand(Bernoulli(p),nqubit(s1))` and flipping the bit when a `1` occurs in the random variable via an `if` statement. But this did not work with the `derivative_estimate()` or `stochastic_triple()` functions and gave error messages. According to the `StochasticAD` documentation, it does not support randomness generated by `if` statements with discrete random input. I tried other variations for bitflips bypassing applying the bitflips via the `if` statement logic, but these got errors as well when interacting with `derivative_estimate()` or `stochastic_triple()`. Below we show another attempt to try to get the `stochastic_triple` to interact with the inner product as we would hope while avoiding the `if` statement logic:

In [6]:
function noisyoverlap(p)

    function bitflips(p) #generates iid bitflips with probability p
    index_list = zeros(nqubits(s1)+1)
    for i in 1:nqubits(s1)
        index_list[i+1] = index_list[i] + rand(Bernoulli(p))
    end
    final = unique(index_list)
    return final[final .> 0] # indices of qubits to be flipped
    end

    temp = copy(s1)
    for item in bitflips(p)
        apply!(temp,sX(item))
    end
    final = dot(temp,s2)
    return final
end

noisyoverlap (generic function with 1 method)

The following call of `stochastic_triple(noisyoverlap,.2)` will give an error:

In [7]:
stochastic_triple(noisyoverlap,.2)

LoadError: MethodError: no method matching Float64(::StochasticAD.StochasticTriple{StochasticAD.Tag{typeof(noisyoverlap), Float64}, Float64, StochasticAD.PrunedFIsModule.PrunedFIs{Float64}})
[0mClosest candidates are:
[0m  (::Type{T})(::Real, [91m::RoundingMode[39m) where T<:AbstractFloat at rounding.jl:200
[0m  (::Type{T})(::T) where T<:Number at boot.jl:772
[0m  (::Type{T})([91m::AbstractChar[39m) where T<:Union{AbstractChar, Number} at char.jl:50
[0m  ...

However, we can say in either the case of bitflip noise or depolariation noise, that these types of noise simply change the signs of some of the generators of a stabilizer state, as conjugation of a Pauli operator by other, different Pauli operators simply changes the sign. If two stabilizer states $S(\ket{\phi}),S(\ket{\psi})$ share a common generator but with opposite signs, their inner product is $0$, and otherwise is $2^{(-s/2)}$, where $s$ is the minimum, over all sets of generators $ { \{P_1, ..., P_n\}} $ for $ S(\ket{\phi}) $ and $ { \{Q_1, ..., Q_n\}} $ for $ S(\ket{\psi}) $, of the different $ i $ values where $ Q_i \neq P_i $. So (roughly), if $p$ varies, we may expect the value of the inner product to change as there are more or less bitflips, thereby in principle allowing for a non-zero value of the derivative `d𝔼[dot(s1,s2)]/dp`. So this would be nice to fix.....